## Sample generation
#### load trained model -> generate samples

In [1]:
import numpy as np
from tensorflow import keras

input_path = 'D:\\Studia\\SEM6\\biai\\projekt\\pretrained_models\\'
input_file_name = 'beethoven50epoch_no_transpositions_512.h5'

model = keras.models.load_model(input_path + input_file_name)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


### Generate seed

In [2]:
import numpy as np

INPUT_SIZE = 256
# [batch_size, sequence_len, input_dim]
# generate random seed
seed = np.random.random((1, 1, INPUT_SIZE))

### Generate sequence

In [3]:
# at default 1 frame = 0.025s
SEQUENCE_LENGTH = 100

x = seed
for _ in range(SEQUENCE_LENGTH):
    print(x.shape)
    res = model.predict(x, batch_size=1)
    x = np.concatenate([seed, res], axis=1)
    
res.shape

(1, 1, 256)
(1, 2, 256)
(1, 3, 256)
(1, 4, 256)
(1, 5, 256)
(1, 6, 256)
(1, 7, 256)
(1, 8, 256)
(1, 9, 256)
(1, 10, 256)
(1, 11, 256)
(1, 12, 256)
(1, 13, 256)
(1, 14, 256)
(1, 15, 256)
(1, 16, 256)
(1, 17, 256)
(1, 18, 256)
(1, 19, 256)
(1, 20, 256)
(1, 21, 256)
(1, 22, 256)
(1, 23, 256)
(1, 24, 256)
(1, 25, 256)
(1, 26, 256)
(1, 27, 256)
(1, 28, 256)
(1, 29, 256)
(1, 30, 256)
(1, 31, 256)
(1, 32, 256)
(1, 33, 256)
(1, 34, 256)
(1, 35, 256)
(1, 36, 256)
(1, 37, 256)
(1, 38, 256)
(1, 39, 256)
(1, 40, 256)
(1, 41, 256)
(1, 42, 256)
(1, 43, 256)
(1, 44, 256)
(1, 45, 256)
(1, 46, 256)
(1, 47, 256)
(1, 48, 256)
(1, 49, 256)
(1, 50, 256)
(1, 51, 256)
(1, 52, 256)
(1, 53, 256)
(1, 54, 256)
(1, 55, 256)
(1, 56, 256)
(1, 57, 256)
(1, 58, 256)
(1, 59, 256)
(1, 60, 256)
(1, 61, 256)
(1, 62, 256)
(1, 63, 256)
(1, 64, 256)
(1, 65, 256)
(1, 66, 256)
(1, 67, 256)
(1, 68, 256)
(1, 69, 256)
(1, 70, 256)
(1, 71, 256)
(1, 72, 256)
(1, 73, 256)
(1, 74, 256)
(1, 75, 256)
(1, 76, 256)
(1, 77, 256)
(1, 78, 

(1, 100, 256)

### Save to midi format

In [17]:
# from midi_numpy.common import save_numpy_midi
from midi_numpy.numpy_to_midi import pipe_reverse as numpy2midi
from datetime import datetime

tim = str(datetime.now()).replace(' ', '_').replace(':', '_')
output_path = 'D:\\Studia\\SEM6\\biai\\projekt\\pretrained_models\\'
output_file_name = f'sample_{SEQUENCE_LENGTH}_{tim}.mid'

# save_numpy_midi(, np.squeeze(res, axis=0))
if len(res.shape) == 3:
    res = np.squeeze(res, axis=0)
res = res.round()
res.shape

print(res[:, :128].max())
print(res.shape)
mid = numpy2midi(res)
mid.tracks[0].append(Message('note_on', note=64, velocity=127, time=123))
mid.tracks[0].append(Message('note_on', note=64, velocity=0, time=123))
print(mid)
mid.save(output_path + output_file_name)

0.0
(100, 256)
<midi file None type 1, 1 tracks, 2 messages>
